In [146]:
from visualize_c3ltc import *

In [368]:
class Square:
    def __init__(self,G,a,g,b):
        square = self.canonical_square(G,a,g,b)
        self.a = square[0]
        self.g = square[1]
        self.b = square[2]
        
    def canonical_square(self,G,a,g,b):
        element_list = list(G)
        option1 = str((a,g,b))
        option2 = str((a.inverse(),a*g,b))
        option3 = str((a,g*b,b.inverse()))
        option4 = str((a.inverse(),a*g*b,b.inverse()))
        minimal = min(option1,option2,option3,option4)
        if minimal == option1:
            return (a,g,b)
        if minimal == option2:
            return (a.inverse(),a*g,b)
        if minimal == option3:
            return (a,g*b,b.inverse())
        if minimal == option4:
            return (a.inverse(),a*g*b,b.inverse())
        
    def __repr__(self):
        return "Square(%s, %s, %s)" % (self.a, self.g, self.b)
    def __eq__(self, other):
        if isinstance(other, Square):
            return (self.a == other.a and self.b == other.b and self.g == other.g)
        else:
            return False
    def __ne__(self, other):
        return (not self.__eq__(other))
    def __hash__(self):
        return hash(self.__repr__())
        
class LeftEdge:
    def __init__(self,G,a,g):
        edge = self.canonical_right_edge(G,a,g)
        self.a = edge[0]
        self.g = edge[1]
        
    def canonical_right_edge(self,G,a,g):
        element_list = list(G)
        index_g = element_list.index(g)
        index_ag = element_list.index(a*g)
        min_index = min(index_g,index_ag)
        if min_index == index_g:
            return (a,g)
        if min_index == index_ag:
            return (a.inverse(),a*g)
    
    def __repr__(self):
        return "LeftEdge(%s, %s)" % (self.a, self.g)
    def __eq__(self, other):
        if isinstance(other, LeftEdge):
            return (self.a == other.a and self.g == other.g)
        else:
            return False
    def __ne__(self, other):
        return (not self.__eq__(other))
    def __hash__(self):
        return hash(self.__repr__())

class RightEdge:
    def __init__(self,G,g,b):
        edge = self.canonical_left_edge(G,g,b)
        self.g = edge[0]
        self.b = edge[1]
    
    def canonical_left_edge(self,G,g,b):
        element_list = list(G)
        index_g = element_list.index(g)
        index_gb = element_list.index(g*b)
        min_index = min(index_g,index_gb)
        if min_index == index_g:
            return (g,b)
        if min_index == index_gb:
            return (g*b, b.inverse())
    
    def __repr__(self):
        return "RightEdge(%s, %s)" % (self.g, self.b)
    def __eq__(self, other):
        if isinstance(other, RightEdge):
            return (self.g == other.g and self.b == other.b)
        else:
            return False
    def __ne__(self, other):
        return (not self.__eq__(other))
    def __hash__(self):
        return hash(self.__repr__())

In [458]:
import numpy
import random

from sage.coding.grs_code import ReedSolomonCode
from sage.rings.finite_rings.finite_field_prime_modn import FiniteField_prime_modn as GF
from sage.groups.perm_gps.permgroup_named import PSL
from sage.matrix.matrix_space import MatrixSpace
from sage.coding.linear_code import LinearCode
from sage.modules.vector_modn_dense import vector


def tensor_decoding(tensor_word, code_a, code_b):
    """ Returns a decoded word in tensor code. 

    Keyword arguments:
    tensor_word -- matrix of length(code_a) x length(code_b).
    code_a -- Sage code object.
    code_b -- Sage code object.
    """

    n_a = len(code_a.parity_check_matrix().columns())
    n_b = len(code_b.parity_check_matrix().columns())
    field = code_a.base_field()

    def tensor_word_to_tuple(m):
        return tuple(m.reshape((n_a * n_b)))

    corrected_word = numpy.copy(tensor_word)
    iterating_set_code_a = [i for i in range(n_a)]
    iterating_set_code_b = [i for i in range(n_b)]
    past_words = []
    init = False
    word_to_tuple = tensor_word_to_tuple(corrected_word)
    while (not init or (
            len(iterating_set_code_a) != 0 or len(iterating_set_code_b) != 0)) and word_to_tuple not in past_words:
        past_words.append(word_to_tuple)
        if not init:
            init = True
        new_iterating_code_a = []
        new_iterating_code_b = []
        for i in iterating_set_code_a:
            local_word = vector(field, corrected_word[i, :])
            try:
                corrected_localy = code_b.decode_to_code(local_word)
            except:
                new_iterating_code_a.append(i)
                continue
            for j in range(n_b):
                if local_word[j] != corrected_localy[j]:
                    new_iterating_code_b.append(j)
                corrected_word[i][j] = corrected_localy[j]
        for j in iterating_set_code_b:
            local_word = vector(field, corrected_word[:, j])
            try:
                corrected_localy = code_a.decode_to_code(local_word)
            except:
                new_iterating_code_b.append(j)
                continue
            for i in range(n_a):
                if local_word[i] != corrected_localy[i]:
                    new_iterating_code_a.append(i)
                corrected_word[i][j] = corrected_localy[i]
        iterating_set_code_a = set(new_iterating_code_a)
        iterating_set_code_b = set(new_iterating_code_b)
        word_to_tuple = tensor_word_to_tuple(corrected_word)
    return corrected_word

def embedding_local_parity_constraints_on_squares(code_a, code_b, G, A, B):
    """ Returns
    1) Sparse representation of the constrsints by amapping of squares (represeted by 3 group elements (a,g,b))
        to dictionary whose keys are rows in which the square has non zero value, and value is the value in the relevant row and column.
    2) Number constraints of rows in the constraint matrix.

    Keyword arguments:
    code_a -- Sage code object.
    code_b -- Sage code object.
    G -- Sage group object.
    A -- list of group elements.
    B -- list of group elements.

    Note:
    1) The length of code_a codewords has to be equal to the number of elements in A.
    2) The length of code_b codewords has to be equal to the number of elements in B.
    3) Both A and B has to be inverse closed (not checked).
    4) code_a and code_b must be defined over the same finite field.
    
    The function performs two stages:
    1) In the first stage, it collects the edges (by A,B) into sets.
    2) In the seocnd stage, the local constraint on each edge are injected according to the squares.
        - Each edges "sees" a tuple of values squares such that the values on those squares 
          should be contained in either code_A or code_B (depending on whether it's an A-edge or B-edge).
        - To enforce the condition above, each one of these squares is associated with a column from the
          parity check of the local code (code_a or code_b). The squares indicate the specific squares that
          participate in the constraints on that edge. 
        - The dictionary (mapping) ''constraints'' in the code, holds a sparse representation of the constraints
          in the parity check induced by the input parameters. It maps a square object (that indicates a column) 
          to a dictionary whose keys are row numbers and the values are the values in the large parity check matrix. 
        - In other words, a copy of the local parity check is being injected into the squares-parity-check
          such that each column of the small parity check is placed into the column associated with different square
          (according to the squares specified by the row). 

    """ 

    constraints = {}
    row = 0
    parity_A = code_a.parity_check_matrix()
    parity_B = code_b.parity_check_matrix()
    codim_C_A = len(parity_A.rows())
    codim_C_B = len(parity_B.rows())

    
    # Stage 1 - collecting the edges
    edges_A = set()
    edges_B = set()
    for g in G:
        for a in A:
            edges_A.add(LeftEdge(G,a,g))
        for b in B:
            edges_B.add(RightEdge(G,g,b))    
            
    assert len(edges_A) == len(list(G)) * len(A) / 2
    assert len(edges_B) == len(list(G)) * len(B) / 2
    
    # Stage 2 - iterating over edges and "injecting" constraints into squares
    for e in edges_A:
        a = e.a
        g = e.g
        for (j, b) in enumerate(B): # B[j] = b
            square = Square(G, a, g, b)
            if square not in constraints: # if no contraints were added on this square before
                constraints[square] = {}
            for (k, v) in enumerate(parity_A[:, j]): # parity_A[k,j] = v
                constraints[square][row + k] = v[0] # v is represted as a length 1 array
        row += codim_C_A
    
    for e in edges_B:
        g = e.g
        b = e.b
        for (i, a) in enumerate(A): # A[i] = a
            square = Square(G, a, g, b)
            if square not in constraints: # if no contraints were added on this square before
                constraints[square] = {}
            for (k, v) in enumerate(parity_B[:, i]): # parity_B[k,i] = v
                constraints[square][row + k] = v[0] # v is represted as a length 1 array
        row += codim_C_B
    
    assert len(constraints) == len(A) * len(B) * len(list(G)) / 4
    assert row == codim_C_A * len(edges_A) + codim_C_B * len(edges_B)
    
    return (constraints, row)

def random_generators(G, n):
    """ Returns an inverse closed set of N elements from G.


    Keyword arguments:
    G -- Sage group object.
    n -- number.

    Note:
    1) n has to be smaller than the number of elements in G.
    """

    list_G = list(G)
    assert n < len(list_G)
    gens = []
    i = 0
    while i < n / 2:
        c = random.choice(list_G)
        if c not in gens and c * c != G.identity():
            gens.append(c)
            gens.append(c.inverse())
            i += 1
    return gens


class c3LTC:

    def __init__(self, code_a, code_b, G, A, B):
        assert len(code_a.generator_matrix().columns()) == len(A)
        assert len(code_b.generator_matrix().columns()) == len(B)
        assert code_a.base_field().characteristic() == code_b.base_field().characteristic()

        (sparse_constraints, count) = embedding_local_parity_constraints_on_squares(code_a, code_b, G, A, B)

        # process sparse constraints

        constraints = numpy.zeros((count, len(sparse_constraints)))
        for (i, l) in enumerate(sparse_constraints):
            for k in sparse_constraints[l]:
                constraints[k][i] = sparse_constraints[l][k]

        # additional mappings

        self.__square_to_index = {}
        self.__index_to_square = {}
        self.squares = list(sparse_constraints)
        self.__list_G = list(G)
        for (i, l) in enumerate(self.squares):
            self.__square_to_index[l] = i
            self.__index_to_square[i] = l
        self.__constraint_matrix = constraints
        self.vertex_to_squares = {}
        for g in G:
            view = numpy.zeros((len(A), len(B)))
            for (i, a) in enumerate(A):
                for (j, b) in enumerate(B):
                    view[i][j] = self.squares.index(Square(G,a, g, b))
            self.vertex_to_squares[self.__list_G.index(g)] = view

        self.edges_A = []
        self.unique_edges_A = []
        self.vertex_to_neighbours_left = {}
        for g in G:
            view = []
            for (i, a) in enumerate(A):
                view.append(self.__list_G.index(a * g))
                self.edges_A.append((a, g))
                if a.inverse() not in A[:i]:
                    self.unique_edges_A.append((a,g))
            self.vertex_to_neighbours_left[self.__list_G.index(g)] = view
        
        self.edges_B = []
        self.unique_edges_B = []
        self.vertex_to_neighbours_right = {}
        for g in G:
            view = []
            for (j, b) in enumerate(B):
                view.append(self.__list_G.index(g * b))
                self.edges_B.append((g, b))
                if b.inverse() not in B[:j]:
                    self.unique_edges_B.append((g,b))
            self.vertex_to_neighbours_right[self.__list_G.index(g)] = view
        self.square_to_vertices = {}
        for (i, s) in enumerate(self.squares):
            view = []
            a = s.a
            g = s.g
            b = s.b
            view.append(self.__list_G.index(a * g))
            view.append(self.__list_G.index(g * b))
            view.append(self.__list_G.index(a * g * b))
            view.append(self.__list_G.index(g * b))
            self.square_to_vertices[i] = view

        # properties of the code

        self.A = A
        self.B = B
        self.code_a = code_a
        self.code_b = code_b
        self.G = G
        self.base_field = code_a.base_field()
        M = MatrixSpace(self.base_field, constraints.shape[0],
                        constraints.shape[1], sparse=True)
        
        dual = LinearCode(M(constraints))
        self.generator_matrix = M(constraints).right_kernel().basis_matrix()
        self.parity_check_matrix = M(constraints)
        self.length = numpy.array(self.generator_matrix).shape[1]
        self.dimension = numpy.array(self.generator_matrix).shape[0]

    def decode_via_edges(self, noisy_word):
        squares_to_values = {}
        for (i, v) in enumerate(noisy_word):
            squares_to_values[self.__index_to_square[i]] = v
        init = False
        iterating_set_A = None
        iterating_set_B = None
        past_words = []
        word_from_square_values = self.__square_to_value_to_word(squares_to_values)
        while (not init or (
                len(iterating_set_A) != 0 or len(iterating_set_B) != 0)) and word_from_square_values not in past_words:
            past_words.append(word_from_square_values)
            if not init:
                init = True
                iterating_set_A = set(self.unique_edges_A)
                iterating_set_B = set(self.unique_edges_B)
            new_iterating_set_A = set([])
            new_iterating_set_B = set([])
            for k,e in enumerate(iterating_set_A):
                a = e[0]
                g = e[1]
                i = self.A.index(a)
                local_word = vector(self.base_field, [0] * len(self.B))
                for (j, b) in enumerate(self.B):
                    square = Square(self.G,a, g, b)
                    local_word[j] = squares_to_values[square]
                try:
                    corrected_localy = self.code_b.decode_to_code(local_word)
                except:
                    new_iterating_set_A.add((a, g))
                    continue
                for (j, b) in enumerate(self.B):
                    square = Square(self.G,a, g, b)
                    if squares_to_values[square] != corrected_localy[j]:
                        new_iterating_set_A.add((a, g * b))
                        new_iterating_set_B.add((g, b))
                        new_iterating_set_B.add((a * g, b))
                    squares_to_values[square] = corrected_localy[j]
                if e in new_iterating_set_A:
                    new_iterating_set_A.remove(e)
            for k,e in enumerate(iterating_set_B):
                g = e[0]
                b = e[1]
                j = self.B.index(b)
                local_word = vector(self.base_field, [0] * len(self.A))
                for (i, a) in enumerate(self.A):
                    square = Square(self.G,a, g, b)
                    local_word[i] = squares_to_values[square]
                try:
                    corrected_localy = self.code_a.decode_to_code(local_word)
                except:
                    new_iterating_set_B.add((g, b))
                    continue
                for (i, a) in enumerate(self.A):
                    square = Square(self.G,a, g, b)
                    if squares_to_values[square] != corrected_localy[i]:
                        new_iterating_set_B.add((a * g, b))
                        new_iterating_set_A.add((a, g))
                        new_iterating_set_A.add((a, g * b))
                    squares_to_values[square] = corrected_localy[i]
                if e in new_iterating_set_B:
                    new_iterating_set_B.remove(e)
            iterating_set_A = new_iterating_set_A
            iterating_set_B = new_iterating_set_B
            word_from_square_values = self.__square_to_value_to_word(squares_to_values)

        return word_from_square_values

    def decode_via_vertices(self, noisy_word):
        n_a = len(self.code_a.parity_check_matrix().columns())
        n_b = len(self.code_b.parity_check_matrix().columns())
        M = MatrixSpace(self.base_field, n_a, n_b)
        squares_to_values = {}
        for (i, v) in enumerate(noisy_word):
            squares_to_values[self.__index_to_square[i]] = v
        init = False
        iterating_set = None
        past_words = []
        word_from_square_values = self.__square_to_value_to_word(squares_to_values)
        while (not init or len(iterating_set)) != 0 and word_from_square_values not in past_words:
            past_words.append(word_from_square_values)
            if not init:
                init = True
                iterating_set = set(self.G)
            new_iterating_set = set([])
            for g in iterating_set:
                local_view = numpy.zeros((n_a, n_b))
                for i, a in enumerate(self.A):
                    for j, b in enumerate(self.B):
                        square = Square(self.G,a, g, b)
                        local_view[i][j] = squares_to_values[square]
                try:
                    corrected_local_view = tensor_decoding(M(local_view), self.code_a, self.code_b)
                except:
                    new_iterating_set.append(g)
                    continue
                for i, a in enumerate(self.A):
                    for j, b in enumerate(self.B):
                        square = Square(self.G,a, g, b)
                        if corrected_local_view[i][j] != local_view[i][j]:
                            new_iterating_set.add(a * g)
                            new_iterating_set.add(g * b)
                            new_iterating_set.add(a * g * b)
                        squares_to_values[square] = corrected_local_view[i][j]
                if g in new_iterating_set:
                    new_iterating_set.remove(g)
            iterating_set = new_iterating_set
            word_from_square_values = self.__square_to_value_to_word(squares_to_values)
        return word_from_square_values

    def __square_to_value_to_word(self, squares_to_values):
        corrected_word = vector(self.base_field, [0] * len(squares_to_values))
        for square in self.__square_to_index:
            corrected_word[self.__square_to_index[square]] = squares_to_values[square]
        return corrected_word

    def syndrome(self, c):
        return self.parity_check_matrix * c

    def local_codeword_on_vertex(self, vertex, word):
        labels_view = self.vertex_to_squares[vertex]
        rows = len(labels_view)
        cols = len(labels_view[0])
        local_view_values = numpy.array([0] * rows
                                        * cols).reshape((rows, cols))
        for i in range(rows):
            for j in range(cols):
                local_view_values[i][j] = int(word[int(self.vertex_to_squares[vertex][i][j])])
        return local_view_values

    def __repr__(self):
        rep = 'c3LTC'
        return rep


In [459]:
G = PSL(2,7)
C_a = ReedSolomonCode(GF(7), Integer(6), Integer(4))
C_b = ReedSolomonCode(GF(7), Integer(6), Integer(4))
A = random_generators(G,6)
B = random_generators(G,6)
c3ltc = c3LTC(C_a, C_b, G, A,B)

In [460]:
import random
for i in range(1):
    word = c3ltc.decode_via_edges(vector(GF(7), [random.randint(0,6)*random.randint(0,1)*random.randint(0,1) for _ in range(c3ltc.length)]))
    print(numpy.count_nonzero(c3ltc.syndrome(word) ) == 0)
    word = c3ltc.decode_via_vertices(vector(GF(7), [random.randint(0,6)*random.randint(0,1)*random.randint(0,1) for _ in range(c3ltc.length)]))
    print(numpy.count_nonzero(c3ltc.syndrome(word) ) == 0)

True
True


In [461]:
show_square(c3ltc,1)

,vertex,left,right,closing
1,100,84,56,84


In [462]:
local_view(c3ltc,1)

,75,166,70,8,108,45
57,1390,773,1391,1187,1251,1132
113,1056,880,1459,1456,1444,1460
149,388,952,953,954,955,357
114,377,783,681,1400,1101,1004
12,997,642,1449,1208,830,114
27,1417,116,807,631,726,1477


In [463]:
local_view_in_word(c3ltc,word,1)

,75,166,70,8,108,45
57,0,0,0,0,0,0
113,0,0,0,0,0,0
149,0,0,0,0,0,0
114,0,0,0,0,0,0
12,0,0,0,0,0,0
27,0,0,0,0,0,0


In [466]:
show_common_views_in_word(c3ltc,word,1,c3ltc.vertex_to_neighbours_left[1][0],"left")

,75,166,70,8,108,45
57,0,0,0,0,0,0
113,0,0,0,0,0,0
149,0,0,0,0,0,0
114,0,0,0,0,0,0
12,0,0,0,0,0,0
27,0,0,0,0,0,0
,131,54,126,64,164,101
113,0,0,0,0,0,0
1,0,0,0,0,0,0
147,0,0,0,0,0,0


In [ ]:
from pyvis import network as net
import networkx as nx
graph = net.Network(notebook=True)
nxg = nx.Graph()

for i,g in enumerate(c3ltc.G):
    nxg.add_node(str(g), label = str(g))

for i,e in enumerate(c3ltc.edges_A):
    nxg.add_node(str(e), label = str(e), color = "blue")

for i,e in enumerate(c3ltc.edges_B):
    nxg.add_node(str(e), label = str(e), color = "red")

for i,s in enumerate(c3ltc.squares):
    nxg.add_node(str(s), label = str(s), color = "orange")

for i,s in enumerate(c3ltc.squares):
    a = s[0]
    g = s[1]
    b = s[2]
    
    e_1 = (a,g)
    e_1_2 = (a.inverse(),a*g)
    e_2 = (g,b)
    e_2_2 = (g*b, b.inverse())
    e_3 = (a,g*b)
    e_3_2 = (a.inverse(),a*g*b)
    e_4 = (a*g,b)
    e_4_2 = (a*g*b, b.inverse())
    
    nxg.add_edge(str(g), str(e_1))
    nxg.add_edge(str(g), str(e_1_2))
    nxg.add_edge(str(g), str(e_2))
    nxg.add_edge(str(g), str(e_2_2))
    
    nxg.add_edge(str(a*g), str(e_1))
    nxg.add_edge(str(a*g), str(e_1_2))
    nxg.add_edge(str(a*g), str(e_4))
    nxg.add_edge(str(a*g), str(e_4_2))
    
    nxg.add_edge(str(g*b), str(e_2))
    nxg.add_edge(str(g*b), str(e_2_2))
    nxg.add_edge(str(g*b), str(e_3))
    nxg.add_edge(str(g*b), str(e_3_2))
    
    nxg.add_edge(str(a*g*b), str(e_3))
    nxg.add_edge(str(a*g*b), str(e_3_2))
    nxg.add_edge(str(a*g*b), str(e_4))
    nxg.add_edge(str(a*g*b), str(e_4_2))
    
    nxg.add_edge(str(e_1), str(s))
    nxg.add_edge(str(e_1_2), str(s))
    nxg.add_edge(str(e_2), str(s))
    nxg.add_edge(str(e_2_2), str(s))
    nxg.add_edge(str(e_3), str(s))
    nxg.add_edge(str(e_3_2), str(s))
    nxg.add_edge(str(e_4), str(s))
    nxg.add_edge(str(e_4_2), str(s))

graph.from_nx(nxg)
graph.show("example.html")

In [195]:
s = Square(G,list(G)[0],list(G)[1],list(G)[2])
s2 = Square(G,list(G)[0],list(G)[1],list(G)[2])

In [196]:
s == s2

True

In [201]:
x = numpy.eye(5)
x[1:3,1] = [1,2]
x

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 2., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])